In [20]:
import numpy as np
import pandas as pd
import pystan
from scipy.stats import norm, multivariate_normal, invwishart, invgamma, bernoulli
from scipy.special import expit
from statsmodels.tsa.stattools import acf


import datetime
import sys
import os

from codebase.plot import * 
from codebase.post_process import * 
from codebase.file_utils import save_obj, load_obj

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data = dict()
df = pd.read_csv("../dat/LSAT.csv")
data['D'] = df.values
data['N'] = df.shape[0]
data['J'] = df.shape[1]
data['K'] = 2

In [3]:
stan_data = dict(N = data['N'], K = data['K'], J = data['J'], DD = data['D'])


In [4]:
with open('./codebase/stan_code/model6_aug.stan', 'r') as file:
    model_code = file.read()


In [5]:
sm = pystan.StanModel(model_code=model_code, verbose=False)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7af366de083ee8d4a8b307ab27daa81d NOW.


In [6]:
nowstr = datetime.datetime.now().strftime('%Y%m%d_%H%M%S_') # ISO 8601 format
nowstr

'20190325_163235_'

In [7]:
task_id = 'LSAT2'
log_dir =  "./log/"+nowstr+"%s/" % task_id
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [8]:
num_chains = 1
num_samples = 1000
num_warmup = 1000
num_iter = num_samples + num_warmup

In [9]:
fit = sm.sampling(data=stan_data, iter=num_iter, chains=num_chains)


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [10]:
save_obj(sm, 'sm', log_dir)
save_obj(fit, 'fit', log_dir)


/Users/itemgmt/myPhD/factor-analysis/src/codebase/file_utils.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


In [11]:
param_names = ['Omega', 'uu', 'beta', 'mu']

stan_samples= fit.extract(permuted=False, pars=param_names)  # return a dictionary of arrays

if num_chains ==1:
    ps = dict()
    for name in param_names:
        ps[name] = np.squeeze(stan_samples[name])
else: 
    ps = stan_samples

In [12]:
# Check that u's cover zero
get_non_zeros(ps['uu'])

(array([], dtype=int64), array([], dtype=int64))

In [13]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['J']):
            plots.append(plot_trace(ps['Omega'][:,i,j],
                     title = 'Posterior distribution for Omega(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_0_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_0_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_0_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_1_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_1_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_1_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_2_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_2_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_2_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_2_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_2_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_3_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_3_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_3_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_3_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_3_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_4_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_4_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_4_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_4_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Omega_left_parenthesis_4_comma_4_right_parenthesis :Curve   [x]   (y)

In [21]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['J']):
            plots.append(plot_trace(acf(ps['Omega'][:,i,j]),
                     title = 'Autocorrelation of Omega(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_4_right_parenthesis :Curve   [x]   (y)

In [22]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
            plots.append(plot_trace(ps['mu'][:,i],
                     title = 'Posterior distribution for mu(%s)'%(i)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Posterior_distribution_for_mu_left_parenthesis_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_mu_left_parenthesis_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_mu_left_parenthesis_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_mu_left_parenthesis_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_mu_left_parenthesis_4_right_parenthesis :Curve   [x]   (y)

In [28]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
            plots.append(plot_trace(acf(ps['mu'][:,i]),
                     title = 'Autocorrelation of mu(%s)'%(i)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Autocorrelation_of_mu_left_parenthesis_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_mu_left_parenthesis_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_mu_left_parenthesis_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_mu_left_parenthesis_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_mu_left_parenthesis_4_right_parenthesis :Curve   [x]   (y)

In [24]:
gamma = np.empty((num_samples, data['J'], data['K']))

for i in range(num_samples):
    e_set = get_topn_eig(ps['beta'][i] @ ps['beta'][i].T,
                         data['K'])
    gamma[i] = e_set['P']
#     lambdas[i] = e_set['L']
    
ps['gamma']= gamma
# ps['lambda'] = lambdas

In [25]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['K']):
            plots.append(plot_trace(ps['gamma'][:,i,j],
                     title = 'Posterior distribution for gamma(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_2_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_2_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_3_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_3_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_4_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_gamma_left_parenthesis_4_comma_1_right_parenthesis :Curve   [x]   (y)

In [27]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['K']):
            plots.append(plot_trace(acf(ps['gamma'][:,i,j]),
                     title = 'Autocorrelation of gamma(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Autocorrelation_of_gamma_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_2_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_2_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_3_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_3_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_4_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_gamma_left_parenthesis_4_comma_1_right_parenthesis :Curve   [x]   (y)